# Numerical Method Convergence Analysis

This notebook analyzes how the binomial tree and Monte Carlo methods converge to the analytical Black-Scholes price.

Understanding convergence behavior is crucial for:
- **Accuracy**: Knowing how many steps/paths are needed
- **Efficiency**: Balancing computation time vs precision
- **Reliability**: Ensuring numerical stability

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import numpy as np

from src.core.option_types import Option, OptionType, ExerciseStyle
from src.analysis.convergence import (
    binomial_convergence,
    monte_carlo_convergence,
)

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

In [ ]:
# Define test option
option = Option(
    spot=100.0,
    strike=100.0,
    rate=0.05,
    volatility=0.20,
    time_to_maturity=1.0,
    option_type=OptionType.CALL,
    exercise_style=ExerciseStyle.EUROPEAN
)

print("Test Option:")
print(f"  Type: European {option.option_type.value}")
print(f"  S={option.spot}, K={option.strike}")
print(f"  r={option.rate:.0%}, σ={option.volatility:.0%}, T={option.time_to_maturity}yr")

## Part 1: Binomial Tree Convergence

The binomial tree approximates continuous price evolution with discrete up/down movements.

### Convergence Rate

As the number of steps $n \to \infty$, the binomial tree converges to Black-Scholes. The error decreases approximately as $O(1/n)$, meaning:
- **Doubling steps** roughly halves the error
- **10x more steps** reduces error by ~10x

In [ ]:
# Test convergence with various step counts
steps_list = [5, 10, 25, 50, 100, 200, 500, 1000]

bs_price, bin_prices, bin_errors = binomial_convergence(option, steps_list)

print(f"Black-Scholes Price: ${bs_price:.6f}")
print()
print("Binomial Tree Convergence:")
print(f"{'Steps':<8} {'Price':<12} {'Error':<12} {'|Error|'}")
print("-" * 42)
for steps, price, error in zip(steps_list, bin_prices, bin_errors):
    print(f"{steps:<8} ${price:<11.6f} {error:+.6f}    {abs(error):.6f}")

In [ ]:
# Plot binomial convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Price vs Steps
ax1.plot(steps_list, bin_prices, 'bo-', markersize=6, linewidth=1.5, label='Binomial Price')
ax1.axhline(y=bs_price, color='r', linestyle='--', linewidth=2, 
            label=f'Black-Scholes (${bs_price:.4f})')
ax1.set_xlabel('Number of Steps', fontsize=12)
ax1.set_ylabel('Option Price ($)', fontsize=12)
ax1.set_title('Binomial Tree Price Convergence', fontsize=14)
ax1.legend(fontsize=10)
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)

# Right: Absolute Error vs Steps (log-log)
abs_errors = [abs(e) for e in bin_errors]
ax2.plot(steps_list, abs_errors, 'go-', markersize=6, linewidth=1.5)
ax2.set_xlabel('Number of Steps', fontsize=12)
ax2.set_ylabel('Absolute Error ($)', fontsize=12)
ax2.set_title('Binomial Tree Error Reduction', fontsize=14)
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Add O(1/n) reference line
ref_steps = np.array([10, 1000])
ref_error = abs_errors[1] * (steps_list[1] / ref_steps)  # Scale from first point
ax2.plot(ref_steps, ref_error, 'r--', alpha=0.5, label=r'$O(1/n)$ reference')
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

### Binomial Convergence Insights

1. **Rapid initial convergence**: Going from 5 to 50 steps dramatically reduces error
2. **Diminishing returns**: Going from 500 to 1000 steps provides little additional accuracy
3. **Oscillation**: Errors can oscillate between positive and negative (odd vs even steps)
4. **Practical recommendation**: 100-200 steps is usually sufficient for most applications

## Part 2: Monte Carlo Convergence

Monte Carlo estimates the option price by averaging simulated payoffs.

### Convergence Rate

By the Central Limit Theorem, the standard error decreases as $O(1/\sqrt{n})$:
- **4x more paths** halves the standard error
- **100x more paths** reduces SE by 10x

This is slower than binomial convergence, but Monte Carlo is more flexible for complex payoffs.

In [ ]:
# Test convergence with various path counts
paths_list = [1000, 5000, 10000, 50000, 100000, 500000]

bs_price_mc, mc_prices, mc_std_errors = monte_carlo_convergence(option, paths_list, seed=42)

print(f"Black-Scholes Price: ${bs_price_mc:.6f}")
print()
print("Monte Carlo Convergence:")
print(f"{'Paths':<10} {'Price':<12} {'Std Error':<12} {'Error vs BS'}")
print("-" * 48)
for paths, price, se in zip(paths_list, mc_prices, mc_std_errors):
    error = price - bs_price_mc
    print(f"{paths:<10,} ${price:<11.4f} ±{se:<11.4f} {error:+.4f}")

In [ ]:
# Plot Monte Carlo convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Price vs Paths with error bars
ax1.errorbar(paths_list, mc_prices, yerr=mc_std_errors, 
             fmt='bo-', markersize=6, linewidth=1.5, capsize=4, capthick=1.5,
             label='Monte Carlo ± 1 SE')
ax1.axhline(y=bs_price_mc, color='r', linestyle='--', linewidth=2, 
            label=f'Black-Scholes (${bs_price_mc:.4f})')
ax1.set_xlabel('Number of Paths', fontsize=12)
ax1.set_ylabel('Option Price ($)', fontsize=12)
ax1.set_title('Monte Carlo Price Convergence', fontsize=14)
ax1.legend(fontsize=10)
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)

# Right: Standard Error vs Paths (log-log)
ax2.plot(paths_list, mc_std_errors, 'go-', markersize=6, linewidth=1.5)
ax2.set_xlabel('Number of Paths', fontsize=12)
ax2.set_ylabel('Standard Error ($)', fontsize=12)
ax2.set_title('Monte Carlo Standard Error Reduction', fontsize=14)
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Add O(1/sqrt(n)) reference line
ref_paths = np.array([1000, 500000])
ref_se = mc_std_errors[0] * np.sqrt(paths_list[0] / ref_paths)
ax2.plot(ref_paths, ref_se, 'r--', alpha=0.5, label=r'$O(1/\sqrt{n})$ reference')
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

### Monte Carlo Convergence Insights

1. **Slow convergence**: Need 100x more paths to reduce error by 10x
2. **Error bars shrink**: Confidence intervals narrow with more paths
3. **Random variation**: Unlike binomial, MC prices vary randomly around the true value
4. **Practical recommendation**: 100,000 paths typically gives ~0.03 SE for standard options

## Comparison: Binomial vs Monte Carlo

| Aspect | Binomial Tree | Monte Carlo |
|--------|---------------|-------------|
| Convergence rate | $O(1/n)$ | $O(1/\sqrt{n})$ |
| Error type | Deterministic | Stochastic |
| American options | Yes | Difficult |
| Path-dependent | Limited | Excellent |
| High dimensions | Poor | Excellent |

### When to Use Each

- **Binomial**: American options, simple European options where speed matters
- **Monte Carlo**: Path-dependent options, high-dimensional problems, when you need confidence intervals

## Summary

Both numerical methods converge to the Black-Scholes price:

1. **Binomial tree**: Fast $O(1/n)$ convergence; 100-200 steps usually sufficient
2. **Monte Carlo**: Slower $O(1/\sqrt{n})$ convergence; 100k paths for ~1% precision

The choice depends on the specific application, with binomial preferred for American options and Monte Carlo for complex path-dependent payoffs.